In [43]:
from dotenv import load_dotenv
import os
from pydantic import BaseModel, Field
from langchain_openai   import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain.agents import create_tool_calling_agent, AgentExecutor
from tools import search_tool
from langchain.chains import LLMMathChain
from langchain.tools import Tool

In [63]:
from langchain_core.tools import tool
import pandas as pd

@tool(parse_docstring=True)
def filter_df(column: str, value: str):
    """
    Filter the DataFrame based on a column and value.

    columns can contain city or shop 
    
    Args:
        column (str): The column name to filter by.
        value (str): The value to filter for.
    
    Returns:
        DataFrame: The filtered DataFrame.
    """
    df = pd.read_csv("./data/forecast_results.csv")
    return df[df[column] == value]

# @tool(parse_docstring=True)
# def filter_date(df, date_: str):
#     """
#     Filter a DataFrame to include only rows for a specific date.

#     Validates that the input date is non-empty and matches the 'YYYY-MM-DD' format.
#     Raises a ValueError if the date is missing or incorrectly formatted.

#     Args:
#         df (pd.DataFrame): The DataFrame to be filtered.
#         date_ (str): The date to filter on. Must be in 'YYYY-MM-DD' format.

#     Returns:
#         pd.DataFrame: A DataFrame containing only the rows with the specified date.

#     Raises:
#         ValueError: If the date is empty or not in the correct 'YYYY-MM-DD' format.
#     """
#     if not isinstance(date_, str):
#         raise ValueError("The date must be a string in the format 'YYYY-MM-DD'.")
#     try:
#         pd.to_datetime(date_, format='%Y-%m-%d')
#     except ValueError:
#         raise ValueError("The date must be in the format 'YYYY-MM-DD'.")
#     if not date_:
#         raise ValueError("The date cannot be empty.")

#     df = df[df["Date"]==date_]
#     return df
@tool(parse_docstring=True)
def calculate_error_agg(column = "error"):
    """
    Calculate the aggregate error from the DataFrame.
    Args:
        column (str): The column name to aggregate. Default is "error".
    Returns:
        float: The sum of the specified column.
    """
    df = pd.read_csv("./data/forecast_results.csv")
    return df[column].sum()

@tool(parse_docstring=True)
def get_error_casuing_child(column, value, child_column):
    """
    Get the child causing the error from a DataFrame.
    
    1. filter based upon the column and value
    2. get the name of all the child using `child_column`
    3. sort the error in descending order
    4. return the name of the child with the highest error
    5. if there is no error return "No errors found"
    6. if there are multiple children with the same error, return the first one
    7. if the child_column is not present in the DataFrame, return "Child column not found"
    8. also return the error of the most error child
    Args:
        column (str): The column name to filter by.
        value (str): The value to filter for.
        child_column (str): The column name containing the child names.
    
    Returns:
        str: The name of the child causing the error.
        float: The error value of the child causing the error.
    """
    df = pd.read_csv("./data/forecast_results.csv")

    if child_column not in df.columns:
        return "Child column not found"
    filtered_df = df[df[column] == value]
    if filtered_df.empty:
        return "No errors found emty DataFrame"
    sorted_df = filtered_df.sort_values(by="error", ascending=False)
    if not sorted_df.empty:
        return sorted_df.iloc[0][child_column], float(sorted_df.iloc[0]["error"])
    else:
        return "No errors found after sorting"
    


    return 

In [64]:
from langchain_openai import ChatOpenAI

load_dotenv()

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
    max_retries=2,
    # api_key="...",
    # base_url="...",
    # organization="...",
    # other params...
)

class PydanticModelResponse(BaseModel):
    topic: str
    summary: str
    sources : list[str]
    tools_used: list[str]



parser = PydanticOutputParser(pydantic_object=PydanticModelResponse)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", " You are a helpful assistant that can answer questions about a given topic. \
         you will filter the DataFrame based on the column and value provided. \
         You will use the tools provided to answer the question. \
         If you cannot answer the question, you will say 'I don't know'. "),
        ("human", "{input}"),
        ("placeholder", "{chat_history}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
).partial(format_instructions=parser.get_format_instructions())

chain = prompt | llm | parser | filter_df | calculate_error_agg | get_error_casuing_child


# chain = prompt | llm | parser | filter_df | calculate_error_agg | get_error_casuing_child | filter_date
# Define the tools to be used by the agent
# tools  = [filter_df, calculate_error_agg, get_error_casuing_child, filter_date]
tools  = [filter_df, calculate_error_agg, get_error_casuing_child]



# Create the agent with the LLM, prompt, and tools
agent = create_tool_calling_agent(
    llm=llm,
    prompt=prompt,
    tools = tools
)

agent_executor = AgentExecutor(agent = agent, tools = tools, verbose=True)

In [65]:
response = agent_executor.invoke(
    {
        "input": "What is the aggregated error for all the cities? then get the error causing shop for the city 'Athens' ",
        "chat_history": [],
        "agent_scratchpad": ""
    }
)



> Entering new AgentExecutor chain...

Invoking: `calculate_error_agg` with `{}`


82456.03217486519
Invoking: `get_error_casuing_child` with `{'column': 'city', 'value': 'Athens', 'child_column': 'shop'}`


('shop_3', 16042.03125182094)The aggregated error for all the cities is 82456.03. 

The shop causing the error for the city 'Athens' is 'shop_3' with an error of 16042.03.

> Finished chain.


In [66]:
response["output"]

"The aggregated error for all the cities is 82456.03. \n\nThe shop causing the error for the city 'Athens' is 'shop_3' with an error of 16042.03."

## Using langchian agent - create_pandas_dataframe_agent

This is a great tool to interact with an datframe using prompts  
CAUTION - you have to precalculate evrything in using python code and then you can use this tool just to create a query around the datframe  


 **Security Notice**:
        This agent relies on access to a python repl tool which can execute
        arbitrary code. This can be dangerous and requires a specially sandboxed
        environment to be safely used. Failure to run this code in a properly
        sandboxed environment can lead to arbitrary code execution vulnerabilities,
        which can lead to data breaches, data loss, or other security incidents.
        
        
        Do not use this code with untrusted inputs, with elevated permissions,
        or without consulting your security team about proper sandboxing!
        You must opt-in to use this functionality by setting allow_dangerous_code=True.
        
        
        This agent relies on access to a python repl tool which can execute 
        arbitrary code. This can be dangerous and requires a specially sandboxed 
        environment to be safely used.

**A Python REPL** (Read-Evaluate-Print Loop) is an interactive environment that allows you to execute Python code and see the results immediately. It's a fundamental part of the Python interpreter, providing a way to experiment with code, test ideas, and explore Python's features. 
Here's a breakdown of what happens in a Python REPL: 
Read: The REPL reads the Python code you enter.
Evaluate: It then executes that code.
Print: The result of the execution is displayed.
Loop: The process repeats, allowing you to enter more code and see the results.

In [37]:
from langchain_experimental.agents import create_pandas_dataframe_agent

agent_executor = create_pandas_dataframe_agent(
    llm,
    df_data,
    agent_type="zero-shot-react-description", # or AgentType.OPENAI_FUNCTIONS for OpenAI models
    verbose=True, # Set to True to see the agent's thought process
    return_intermediate_steps=True, # Optional, to get intermediate steps, 
    allow_dangerous_code = True
)

In [42]:
response = agent_executor.invoke(
    {
        "input": "Filter the DataFrame to show only the data of Athens, calulate the total quantity of the athens on Fen'2016 \
            get the brand which is most popular in Athens and \
            summarize the data in a table format. calculate popularity based upon quantity column" ,
        "chat_history": [],
        "agent_scratchpad": ""
    }
)



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to show only the data of Athens and then calculate the total quantity of Athens in February 2016. After that, I need to find the brand which is most popular in Athens based on quantity and summarize the data in a table format.

Action: python_repl_ast
Action Input: df['date'] = pd.to_datetime(df['date'], format='%d/%m/%y')I have converted the 'date' column to datetime format, now I can filter the dataframe based on the city and date.
Action: python_repl_ast
Action Input: athens_data = df[(df['city'] == 'Athens') & (df['date'].dt.month == 2) & (df['date'].dt.year == 2016)]I have filtered the dataframe to show only the data of Athens in February 2016. Now I can proceed to calculate the total quantity and find the most popular brand.
Action: python_repl_ast
Action Input: total_quantity = athens_data['quantity'].sum()I have calculated the total quantity of Athens in February 2016. Now I need to find the most po

In [43]:
response["output"]

'The most popular brand in Athens in February 2016 based on quantity is the one with the highest popularity percentage in the table below:\n\n|        | brand       | quantity | popularity |\n|-------:|:------------|---------:|-----------:|\n|      0 | kinder-cola |     16575 |   0.5525   |\n|      1 | adult-cola  |     13400 |   0.4475   |'